In [2]:
%load_ext autoreload
%autoreload 2

## Part II: Model Building

Here you try your hand at model building to predict appointment no shows.

### Preprocessing

Package 'proj2_lib' now includes code to carry out preprocessing steps from part I. Here's how to use it:

In [3]:
import proj2_lib.util as utils

First, it includes a dictionary used for configuring path and file names
used through the project

In [4]:
utils.file_config

{'feature_pipeline_file': 'feature_pipeline.pkl',
 'labels_pipeline_file': 'labels_pipeline.pkl',
 'objstore_path': 'objects',
 'processed_data_path': 'processed_data',
 'raw_data_csv': 'KaggleV2-May-2016.csv',
 'raw_data_path': 'data',
 'test_csv': 'test_set.csv',
 'train_csv': 'train_set.csv'}

`feature_pipeline_file`: file storing the preprocessing pipeline used for preparing the feature matrix

`labels_pipeline_file`: file storing the preprocessing pipeline used for
preparing labels

`objstore_path`: directory to store python objects to disk

`processed_data_path`: directory containing processed data

`raw_data_csv`: name of the csv download from Kaggle

`raw_data_path`: directory containing raw data

`test_csv`: name of csv file containing test set

`train_csv`: name of csv file containing train set

You can change these paths and names to suit your project directory structure if you need so. E.g.,

In [5]:
file_config = utils.file_config
#config['raw_data_path'] = "some_other_directory"

First step is to create train test sets. Code is in file `proj2_lib/util.py` function `make_train_test_sets`. You
can edit that function as needed to include your own part I code if you so desire. The result will be to 
create files `train_set.csv` and `test_set.csv` in your `processed_data` directory (unless you change any of the entries in the configuration directory as above).

In [6]:
# ONLY NEED TO RUN THIS STEP ONCE (switch this to True to run it)
RUN_MAKE_TRAIN_TEST_FILES = False
if RUN_MAKE_TRAIN_TEST_FILES:
    utils.make_train_test_sets(config=file_config)

Next step is to fit the preprocessing pipelines. This is done in file `proj2_lib/preprocess.py`. Again you can edit code as needed in that file to incorporate your part I solution as you wish. The result will be to create files `feature_pipeline.pkl` and `labels_pipeline.pkl` containing the fit preprocessing pipelines we can then use to preprocess data.

In [7]:
import proj2_lib.preprocess as preprocess

# ONLY NEED TO RUN THIS STEP ONCE
RUN_FIT_PREPROCESSING = False
if RUN_FIT_PREPROCESSING:
    preprocess.fit_save_pipelines(config=file_config)

Finally, once we do that, we can get a training matrix and labels:

In [8]:
train_X, train_y = preprocess.load_train_data(config=file_config)

In [9]:
print(train_X.shape)
print(train_y.shape)

(90526, 101)
(90526,)


### Model Building

Using `sklearn` fit:
    - DecisionTree classifier
    - RandomForest classifier
    - Linear SVM classifier
    - SVM with Radial Basis Kernel classifier
    
Use default parameters for now.
Using 10-fold cross validation report both accuracy and AUC for each of the above four models.

QUESTION: Should you use accuracy or AUC for this task as a performance metric?

- In this case, I believe that AUC would be a better metric since the value is more distictive for each model.

In [10]:
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn import metrics
from sklearn.model_selection import KFold 

In [12]:
names = ["DecisionTreeClassifier","RandomForestClassifier"] #"LinearSVC","RadialSVC"]
classifiers = [DecisionTreeClassifier(random_state=1234),
              RandomForestClassifier(random_state=1234)]#,
              #SVC(kernel='linear',probability=True,random_state=1234),
              #SVC(kernel='rbf',probability=True,random_state=1234)]
   

def getModel_aveAccuracy_aveACU(clf,n_folds,train_X,train_y):
    k_fold = KFold(n_splits=n_folds, shuffle=False, random_state=1234)
    
    cum_auc = cum_score = cum_acc = 0
    for k,(train_idx,test_idx) in enumerate(k_fold.split(train_X)):
        mdl = clf.fit(train_X[train_idx],train_y[train_idx])
        y_pred = mdl.predict(train_X[test_idx])
        prob = mdl.predict_proba(train_X[test_idx])
        fpr, tpr, _ = metrics.roc_curve(train_y[test_idx], prob[:,1])
        
        cum_auc += metrics.auc(fpr,tpr)
        #cum_score += clf.score(train_X[test_idx], train_y[test_idx])
        cum_acc += metrics.accuracy_score(train_y[test_idx],y_pred)
        
    #return cum_auc/n_folds,cum_score/n_folds,cum_acc/n_folds
    return cum_auc/n_folds,cum_acc/n_folds

    
#testing code 
for idx,c in enumerate(classifiers):    
    avg_auc,ave_acc = getModel_aveAccuracy_aveACU(c,10,train_X,train_y)
    print(names[idx],"Avg AUC:",avg_auc,"Avg Accuracy:",ave_acc)

DecisionTreeClassifier Avg AUC: 0.584360484063 Avg Accuracy: 0.738804292579
RandomForestClassifier Avg AUC: 0.690972732343 Avg Accuracy: 0.776539295353


### Model Tuning

Based on the above, choose two methods and fit a tuned model:
    - use 5-fold cross validation for model selection
    - use 10-fold cross validation for model assessment (based on appropriate performance metric)

Report estimated performance for both tuned classifiers

### Linear SVM with Gradient Descent

In [49]:
import numpy as np

# initialize model parameters w and b
# intializing to 0 is not a good idea
# it should be a random vector see np.random.randn
# YOU NEED TO IMPLEMENT THIS
def _initialize_parameters(nfeatures):
    #w = np.full((nfeatures), 0.0)
    #b = np.full((1), 0.0)
    w = np.random.rand(1,(nfeatures))[0]
    b = np.random.rand(1,(1))[0]
    return w, b

# this is a vectorized version of positive_part operation
# we can use this for hinge loss as post_part(1.0 - y*f)
pos_part = np.vectorize(lambda u: u if u > 0. else 0.)

# compute the value of the linear SVM objective function
# given current signed distances, and parameter vector w
def _get_objective(f, y, w, lam):
    loss = np.sum(pos_part(1.0 - y*f))
    penalty = lam * np.dot(w,w)
    return loss + penalty

# compute the signed distances
# based on current model estimates
# w and b
def _get_signed_distances(X, w, b):
    #nobs = X.shape[0]
    #f = np.full(nobs, 0.0)
    f = np.dot(X,w) + b
    return f

# compute gradients with respect to w and b
# ASUMMING exponential loss
def _get_gradients(f, X, y, w, b, lam):
    nfeatures = X.shape[1]
    gw = np.full((nfeatures), 0.)
    #TODO : correct gw and loss function
    #gw = (-1)*sum([(y[i]*X[i]*np.exp((-1)*y[i]*(np.dot(w,X[i])+b))) for i in range(nfeatures)])+lam*w
    gb = (-1)*sum([(y[i]*np.exp((-1)*y[i]*(np.dot(w,X[i])+b))) for i in range(nfeatures)])
    return gw, gb

# fit an SVM using gradient descent
# X: matrix of feature values
# y: labels (-1 or 1)
# n_iter: numer of iterations
# eta: learning rate
def fit_svm(X, y, lam, n_iter=100, eta=.4):
    nexamples, nfeatures = X.shape
    
    w, b = _initialize_parameters(nfeatures)
    
    for k in range(n_iter):
        f = _get_signed_distances(X, w, b)
        
        # print information and 
        # update the learning rate
        if k % 10 == 0:
            obj = _get_objective(f, y, w, lam)
            eta = eta / 2.0
            print("it: %d, obj %.2f" % (k, obj))
        
        gw, gb = _get_gradients(f, X, y, w, b, lam)
        w = w - eta * gw
        b = b - eta * b
    return w, b

In [50]:
w,b = fit_svm(train_X, train_y, 1.0, n_iter=100)

it: 0, obj 273880.07
it: 10, obj 220723.52
it: 20, obj 216622.64
it: 30, obj 215743.65
it: 40, obj 215450.44
it: 50, obj 215330.17
it: 60, obj 215275.64
it: 70, obj 215249.67
it: 80, obj 215237.00
it: 90, obj 215230.74
